# Прогнозирование заказов такси

**Описание проекта**

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах: клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

**Описание данных**

Данные находятся в файле `/datasets/toxic_comments.csv`.

Столбец `text` в нём содержит текст комментария, а `toxic` — целевой признак.

**План работы**
1. Загрузим и подготовим данные.
2. Обучим разные модели.
3. Сделаем выводы.

In [2]:
import pandas as pd
import numpy as np
import string
import optuna
import plotly.express as px

from collections import defaultdict
from IPython.display import display

from fast_ml import eda
from ydata_profiling import ProfileReport

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

from sklearn.utils import resample
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.model_selection import cross_val_score

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\micha\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\micha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
FIG_WIDTH = 10 * 100
FIG_HEIGHT = 5 * 100
RANDOM_SEED = 42

In [4]:
try:
    raw_comments = pd.read_csv('toxic_comments.csv')
except:
    raw_comments = pd.read_csv('/datasets/toxic_comments.csv')

## Исследовательский анализ данных

Изучим основные зависимости в данных перед тем, как мы будем использовать их в алгоритмах машинного обучения.

Таблица-резюме:

In [5]:
display(eda.df_info(raw_comments))

,data_type,data_type_grp,num_unique_values,sample_unique_values,num_missing,perc_missing
Unnamed: 0,int64,Numerical,159292,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]",0,0.0
text,object,Categorical,159292,[Explanation\nWhy the edits made under my user...,0,0.0
toxic,int64,Numerical,2,"[0, 1]",0,0.0


Числовые распределения:

In [6]:
display(round(raw_comments.describe().T, 2))

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,159292.0,79725.7,46028.84,0.0,39872.75,79721.5,119573.25,159450.0
toxic,159292.0,0.1,0.30,0.0,0.00,0.0,0.00,1.0


И детальный отчет:

In [7]:
ProfileReport(raw_comments).to_widgets()

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Выводы

# Подготовка данных для ML

Данные уже достаточно чистые, поэтому обработка будет относительно быстрой:

1. Отбросим первую колонку - она нам не нужна.
2. Причешем текст: приведем его к нижнему регистру, уберем пунктуацию, уберем стоп-слова, лемматизируем слова.

Создадим несколько функций, для обработки текста.

In [8]:
def lowercase_text(text):
    """
    Convert the provided text to lowercase.
    
    Parameters:
    - text (str): The text to be converted.
    
    Returns:
    - str: The text in lowercase.
    """
    return text.lower()

def remove_punctuation(text):
    """
    Remove punctuation from the provided text.
    
    Parameters:
    - text (str): The text from which punctuation should be removed.
    
    Returns:
    - str: The text without punctuation.
    """
    return text.translate(str.maketrans('', '', string.punctuation))

def remove_stopwords(text):
    """
    Remove stopwords from the provided text using NLTK's English stopwords list.
    
    Parameters:
    - text (str): The text from which stopwords should be removed.
    
    Returns:
    - str: The text without stopwords.
    """
    stop_words = set(stopwords.words('english'))
    return ' '.join([word for word in text.split() if word not in stop_words])

def lemmatize_text(text):
    """
    Lemmatize the provided text using NLTK's WordNetLemmatizer.
    
    Parameters:
    - text (str): The text to be lemmatized.
    
    Returns:
    - str: The lemmatized text.
    """
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

Применим эти функции.

In [9]:
df_comments = (
    raw_comments
    .drop('Unnamed: 0', axis=1)
    .assign(text=lambda x: x.text.apply(lowercase_text))
    .assign(text=lambda x: x.text.apply(remove_punctuation))
    .assign(text=lambda x: x.text.apply(remove_stopwords))
    .assign(text=lambda x: x.text.apply(lemmatize_text))
)

display(df_comments.head())

,text,toxic
0,explanation edits made username hardcore metal...,0
1,daww match background colour im seemingly stuc...,0
2,hey man im really trying edit war guy constant...,0
3,cant make real suggestion improvement wondered...,0
4,sir hero chance remember page thats,0


И теперь проверим word-cloud.

In [10]:
ProfileReport(df_comments).to_widgets()

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Выводы

## Модели ML

Создадим и обучим несколько моделей. Для начала разделим данные на `train` и `test` выборки. Создадим функцию для этого, чтобы сразу записать в датафрейм.

In [11]:
def split_data(df: pd.DataFrame, target_column: str, test_size: float, shuffle=False, stratify=None, split_ftr_tgt=True):
    """
    Split a DataFrame into training and testing datasets.

    This function accepts a DataFrame, the name of the target column, and the proportion of the data 
    to be included in the test split. Depending on the 'split_ftr_tgt' parameter, it either returns 
    four DataFrames (training features, training target, testing features, testing target) or two DataFrames 
    (complete training data, complete testing data).

    Args:
    - df (pd.DataFrame):
        The DataFrame to split. This DataFrame should include both the features and the target.

    - target_column (str): 
        The name of the target column. If 'split_ftr_tgt' is True, this column will be separated 
        from the features and returned in the target DataFrames.

    - test_size (float):
        The proportion of the data to include in the test split. For example, if `test_size` is 0.3, 
        30% of the data will be used for the test split, and the rest will be used for the training split.
        
    - shuffle (boolean):
        A flag to shuffle (True) or not (False) the data when splitting into train and test.

    - stratify (array-like):
        Determines the stratification strategy. If not None, data is split in a stratified fashion, using 
        the provided array as the class labels. It ensures that the training and test datasets have 
        approximately the same percentage of samples of each target class as the complete set.

    - split_ftr_tgt (boolean):
        If True, the function will return separate DataFrames for features and target for both training 
        and testing data. If False, it will return complete DataFrames for training and testing data.

    Returns:
    - list of pd.DataFrame:
        Depending on 'split_ftr_tgt':
            If True: A list containing four DataFrames - the training features, the training target, 
                     the testing features, and the testing target.
            If False: A list containing two DataFrames - the complete training data and the complete testing data.
    """
    df_train, df_test = train_test_split(
        df, test_size=test_size, random_state=RANDOM_SEED, shuffle=shuffle, stratify=stratify
    )
    
    if split_ftr_tgt:
        ftr_train = df_train.drop(target_column, axis=1)
        tgt_train = df_train[[target_column]]
        ftr_test = df_test.drop(target_column, axis=1)
        tgt_test = df_test[[target_column]]
        return [ftr_train, tgt_train, ftr_test, tgt_test]
    else:
        return [df_train, df_test]

У нас есть сильный дисбаланс классов в целевой переменной. Попробуем его решить с помощью `upsampling`.

In [12]:
def upsample_data(df, target_column):
    """
    Upsamples the minority class in the dataframe.
    
    Parameters:
    - df: DataFrame containing the data
    - target_column: str, name of the target column
    
    Returns:
    - DataFrame with upsampled minority class
    """
    # Separate the majority and minority classes
    df_majority = df[df[target_column] == 0]
    df_minority = df[df[target_column] == 1]

    # Upsample the minority class
    df_minority_upsampled = resample(
        df_minority, 
        replace=True,                              
        n_samples=len(df_majority),                
        random_state=RANDOM_SEED
    )

    # Combine the majority class with the upsampled minority class
    df_upsampled = pd.concat([df_majority, df_minority_upsampled])

    # Shuffle the dataset to mix the two classes
    df_upsampled = df_upsampled.sample(frac=1, random_state=RANDOM_SEED).reset_index(drop=True)
    
    return df_upsampled

Сохраним выборки, проверим `upsampling`.

In [13]:
def print_class_balance(df, label):
    distribution = df.toxic.value_counts(normalize=True)
    print(f"Class balance {label}: {distribution[1]:.2f} - 1 and {distribution[0]:.2f} - 0")

# Define splits
dct_splits = {
    'train': split_data(df_comments, 'toxic', 0.1, split_ftr_tgt=False)[0],
    'test': split_data(df_comments, 'toxic', 0.1, split_ftr_tgt=False)[1]
}

# Check train-test split
print('Test to full sample size:', round(100 * dct_splits['test'].shape[0] / df_comments.shape[0], 2), '%')

# Upsample and show the class balance
print_class_balance(dct_splits['train'], 'before upsampling')
dct_splits['train'] = upsample_data(dct_splits['train'], 'toxic')
print_class_balance(dct_splits['train'], 'after upsampling')

# Update splits with separated features and target directly
dct_splits = {
    'train': {
        'features': dct_splits['train'].drop('toxic', axis=1),
        'target': dct_splits['train'][['toxic']]
    },
    'test': {
        'features': dct_splits['test'].drop('toxic', axis=1),
        'target': dct_splits['test'][['toxic']]
    }
}

Test to full sample size: 10.0 %
Class balance before upsampling: 0.10 - 1 and 0.90 - 0
Class balance after upsampling: 0.50 - 1 and 0.50 - 0


Выборки разбились как надо. `upsampling` удался.

Теперь проведем токенизацию и воспользуемся `tf-idf` для создания признаков.

In [14]:
 # Consider unigrams and bigrams, limit features to 5000 for computational efficiency
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))

dct_splits['train']['features'] = vectorizer.fit_transform(dct_splits['train']['features']['text'])
dct_splits['test']['features'] = vectorizer.transform(dct_splits['test']['features']['text'])

dct_splits['train']['target'] = dct_splits['train']['target']['toxic']
dct_splits['test']['target'] = dct_splits['test']['target']['toxic']

Зададим `study` для `optuna` - она сделает для нас оптимальные модели.

In [19]:
def optimize_classifiers(ftr_train, tgt_train, n_trials: int):
    """
    Trains and optimizes classifiers using Optuna.

    Args:
    - ftr_train, tgt_train: Training features and target.
    - n_trials (int): The number of trials for Optuna optimization.

    Returns:
    - An Optuna study object containing the optimal model and its parameters.
    """
    # Ensure target is 1-d vector
    tgt_train = tgt_train.values.ravel()

    def get_classifier(trial):
        classifiers = {
            'LogisticRegression': LogisticRegression(
                C=trial.suggest_float('lr_C', 0.001, 1, log=True),
                max_iter=trial.suggest_int('lr_max_iter', 100, 1000, step=100),
                random_state=RANDOM_SEED
            ),
            'DecisionTree': DecisionTreeClassifier(
                max_depth=trial.suggest_int('dt_max_depth', 1, 100),
                random_state=RANDOM_SEED
            ),
            'LightGBM': LGBMClassifier(
                max_depth=trial.suggest_int('lgbm_max_depth', 1, 100),
                n_estimators=trial.suggest_int('lgbm_n_estimators', 100, 1000),
                learning_rate=trial.suggest_float('lgbm_learning_rate', 0.01, 0.3),
                random_state=RANDOM_SEED
            ),
            'CatBoost': CatBoostClassifier(
                iterations=trial.suggest_int('cb_iterations', 400, 1000),
                learning_rate=trial.suggest_float('cb_learning_rate', 0.1, 0.4),
                logging_level='Silent',
                random_state=RANDOM_SEED
            ),
        }
        classifier_name = trial.suggest_categorical('classifier', list(classifiers.keys()))
        return classifiers[classifier_name]

    def objective(trial):
        """
        Objective function for Optuna optimization. Computes the F1-score for a given classifier.

        Args:
        - trial (optuna.Trial): 
            A trial is a process of evaluating an objective function. 
            This object is passed to an objective function and provides interfaces to 
            suggest hyperparameters.

        Returns:
        - float:
            Average F1-score from cross-validation of the classifier's predictions.
        """
        classifier_obj = get_classifier(trial)
        scores = cross_val_score(classifier_obj, ftr_train, tgt_train, scoring='f1', cv=3)
        
        return np.mean(scores)

    optuna.logging.set_verbosity(optuna.logging.WARNING)
    study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=RANDOM_SEED))
    study.optimize(objective, n_trials=n_trials)
    
    return study

Возьмем результаты `study`.

In [20]:
study = optimize_classifiers(
    dct_splits['train']['features'],
    dct_splits['train']['target'],
    n_trials=10
)

Посмотрим, как модели соотносятся друг с другом.

In [21]:
best_params = study.best_params
formatted_params = "\n".join([f"  {key}: {value}" for key, value in best_params.items()])
print(f"Best params:\n{formatted_params}")

fig = optuna.visualization.plot_optimization_history(study)
fig.update_layout(
    legend=dict(orientation='h'),
    template='plotly_white',
    width=FIG_WIDTH, height=FIG_HEIGHT
)
fig.show()

fig = optuna.visualization.plot_slice(study)
fig.update_layout(
    legend=dict(orientation='h'),
    template='plotly_white',
    width=FIG_WIDTH, height=FIG_HEIGHT
)
fig.update_xaxes(tickangle=-90)
fig.show()

Best params:
  lr_C: 0.004857295179217167
  lr_max_iter: 100
  dt_max_depth: 29
  lgbm_max_depth: 17
  lgbm_n_estimators: 937
  lgbm_learning_rate: 0.2443549100736809
  cb_iterations: 780
  cb_learning_rate: 0.3614381770563153
  classifier: LightGBM


Процесс оптимизации показывает, что, учитывая имеющиеся данные и рассматриваемый диапазон гиперпараметров, регрессор `CatBoost` обеспечивает наименьшую ошибку RMSE на обучающих данных. Ожидаем, что эта модель покажет лучшие результаты на неизвестных данных среди всех рассмотренных моделей и гиперпараметров.

## Проверка результатов

Последним этапом, проверим, как модель работает на тестовых данных, а также, какие прогнозы у нас получились.

Для начала обучим лучшую модель.

In [18]:
model = CatBoostRegressor(
    iterations=study.best_params['iterations'],
    learning_rate=study.best_params['learning_rate'],
    logging_level='Silent',
    random_state=RANDOM_SEED
)

model.fit(dct_splits['train']['features'], dct_splits['train']['target']);

NameError: name 'CatBoostRegressor' is not defined

Посчитаем предсказания и посмотрим на метрики.

In [ ]:
for sample in ['train', 'test']:
    # Convert predictions array to DataFrame
    dct_splits[sample]['prediction'] = pd.DataFrame(
        model.predict(dct_splits[sample]['features']),
        columns=['num_orders'],
        index=dct_splits[sample]['target'].index
    )
    
    rmse = np.sqrt(
        mean_squared_error(
            dct_splits[sample]['target'], dct_splits[sample]['prediction']
        )
    )
    
    print(f"RMSE on {sample} sample: {rmse:.3f}")

RMSE on train sample: 15.506
RMSE on test sample: 45.789


И из любопытсва посмотрим на предсказания на графиках.

In [ ]:
df_temp = (
    pd.concat([
        dct_splits[split][data_type]
        .assign(
            split=split, is_predicted=(data_type == 'prediction')
        )
        for split in ['train', 'test']
        for data_type in ['target', 'prediction']
    ])
    .pipe(
        lambda df: df.assign(rolling_3h=df.num_orders.rolling(3).mean())
    )
)

display(df_temp.head())

,num_orders,split,is_predicted,rolling_3h
datetime,,,,
2018-03-01 03:00:00,66.0,train,False,NaN
2018-03-01 04:00:00,43.0,train,False,NaN
2018-03-01 05:00:00,6.0,train,False,38.333333
2018-03-01 06:00:00,12.0,train,False,20.333333
2018-03-01 07:00:00,15.0,train,False,11.000000


In [ ]:
fig_config = {
    'train': {
        'x_range': [df_temp.index.min(), df_temp.index.min() + pd.Timedelta(days=14)],
        'y_range': [0, 150]
    },
    'test': {}
}

for split, config in fig_config.items():
    fig = px.line(
        df_temp[df_temp.split == split].reset_index(),
        x='datetime',
        y='rolling_3h',
        color='is_predicted',
        title=f'True and predicted values for {split} sample',
        template='plotly_white',
        width=FIG_WIDTH, height=FIG_HEIGHT
    )
    fig.update_xaxes(range=config.get('x_range'))
    fig.update_yaxes(range=config.get('y_range'))
    fig.update_layout(legend=dict(orientation='h'))
    fig.show()


## Выводы

На основе проведенного анализа и результатов моделирования можно сделать следующие выводы:

1. Данные, связанные с заказами такси, демонстрируют явную сезонность и растущий тренд, что указывает на успешное развитие бизнеса. Особенно активные заказы наблюдаются в ночные часы.
  
2. Модель `CatBoost` с определенными гиперпараметрами показала наилучшие результаты среди рассмотренных, что подтверждается значением RMSE на обучающих данных.

3. Несмотря на хорошие показатели на обучающей выборке, модель демонстрирует более высокую ошибку на тестовой выборке, что может указывать на переобучение или изменение характера данных.

4. Остаточная составляющая модели показывает увеличение ошибки после августа, что может требовать дополнительного анализа или коррекции модели для этого временного промежутка.

5. В целом, модель может быть использована для прогнозирования заказов такси, но надо аккуратно выбирать горизонт прогнозирования, а также регулярно проверять модель на актуальных данных.